# Рекомендательные системы

Основано на материалах курса Д. И. Игнатова "Рекомендательные системы"

## Что такое рекомендательная система?

У нас есть множество пользователей. Для каждого пользователя есть множество объектов, которые он оценил (поставил рейтинг). Задачей рекомендательной системы является предсказание этого рейтинга для новых для пользователя объектов. Потом объекты с наиболее высокой оценкой будут предложены пользователю в ленте рекомендаций.

<img src="https://d2l.ai/_images/rec-intro.svg" alt="rec" width=800/>

Например, мы можем предсказывать:
- Оценку для фильма на сайте онлайн-кинотеатра
- Оценку для книги на сайте электронной библиотеки
- Просмотрит ли человек короткий ролик до конца
- Прослушает ли до конца песню
- И т.д.

Более формально: для каждой пары пользователь-объект оценить рейтинг и предсказать топ наиболее "релевантных" пользователю объектов.

При этом оценка от пользователя обычно делится на два вида:
- __Явная__, когда пользователь точно выразил свое отношение к объекту (поставил лайк, написал отзыв)
- __Неявная__, когда мы вынуждены по косвенным признакам определять оценки (время просмотра видео, клики)

***

Бывает, что задачу ставят в более общем виде как задачу предсказания наличия ребер в двудольном графе. Тогда мы считаем, что у нас есть:
- Два набора объектов (обычно) различной природы (авторы и статьи, покупатели и продукты, книги и ключевые слова). При этом принято один набор называть объектами, а второй $-$ атрибутами.
- Информация о наличии связей между объектами. Причем связи могут быть только между объектами из разных наборов.


<img src="https://www.researchgate.net/publication/369199718/figure/fig4/AS:11431281183703059@1692966061534/K-partite-graph-illustration-a-A-user-item-bipartite-graph-with-nodes-representing-user.png" width=600 alt="rec">


И здесь есть два варианта постановки задачи:
- Object-Attribute task: предсказываем, должен ли данный объект иметь связь с данным атрибутом (является ли человек автором статьи, купит ли клиент продукт)
- Object-Object task: предсказываем, должны ли два объекта иметь связь с одним атрибутом (могут ли два человека быть соавторами, будут ли два продукта куплены вместе)


## Как выглядят данные для обучения?

Чаще всего данные представляют собрй матрицу, где по строкам и столбцам расположены два набора объектов, а в ячейках стоят оценки:

| user_id <br> item_id | 1 | 2 | 3 | 4 | 5 |
| -- | -- | -- | -- | -- | -- |
| 1 | 5.0 | 0.0 | 0.0 | 3.0 | 3.0 |
| 2 | 4.0 | 0.0 | 0.0 | 0.0 | 0.0 |
| 3 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 |
| 4 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 |
| 5 | 4.0 | 3.0 | 0.0 | 0.0 | 0.0 |


Иногда это длинная форма такой матрицы:

| user_id | item_id | rating |
| -- | -- | -- |
| 1 | 1 | 5.0 |
| 1 | 4 | 3.0 |
| 1 | 5 | 3.0 |
| 2 | 1 | 4.0 |
| 5 | 1 | 4.0 |
| 5 | 2 | 3.0 |

Хорошие датасеты:
- [GroupLens](https://grouplens.org/datasets/movielens/)
- [RecSys Challenge](https://www.recsyschallenge.com/2024/)
- [Kinopoisk's movies reviews](https://www.kaggle.com/datasets/mikhailklemin/kinopoisks-movies-reviews): русскоязычный и подходит для content-based решений
- [Netflix Movies and TV Shows](https://www.kaggle.com/datasets/shivamb/netflix-shows) - англоязычный, подходит для content-based

## Методы оценки

Так как мы фактически решаем две задачи: предсказание наличия связи (и это классификация) и предсказание силы связи $-$ оценки (это регресиия), мы имеем два набора метрик:
1. MSE, MAE etc $-$ для предсказания оценки
2. Precision, Recall, F-score $-$ для предсказания связи

Также мы можем использовать метрики
- для сравнения множеств (коэффициент Жаккара)

$J(A, B) = \LARGE{\frac{\lvert A\cap B \rvert}{\lvert A \cup B \rvert}}$;

- корреляционные оценки (коэффициент Пирсона)

$r = \LARGE{\frac{\sum_{i=1}^n (x_i - \bar{x}) \times (y_i - \bar{y})}{\sqrt{{\sum_{i=1}^n (x_i - \bar{x}) ^ 2} \times {\sum_{i=1}^n (y_i - \bar{y}) ^ 2}}}}$;

- и метрики для ранжирования (precision@k - сколько релевантных элементов оказалось в топ-k лучших предсказаний).



## Подходы

### Content-based

Здесь мы делаем предсказания на основе похожести объектов с точки зрения контента. При этом под контентом может пониматься как реальное соджержание объекта (текст книги / песни, картинка), так и метаинформация о нем (актеры и кассовые сборы, описание картинки или песни).

Обычно у данного подхода выделяют несколько недостатков:
- Большая вычислительная сложность (тяжело работать с музыкой или большими текстами)
- Нужны сложные доменные модели. Мы не можем считать картинку и текст похожими объектами, если у нас нет мультимодальных моделей
- Для многих типов объектов не очень понятно, что такое контент в целом
- Нужно много данных для первичного обучения

### Collaborative filtering

Здесь есть два набора подходов:
1. Item-based: рекомендуем новые объекты на основе их похожести на выбранные пользователем ранее. Какие здесь могут быть сложности? Что значит "похожесть" в данном случае?

2. User-based: рекомендуем пользователю новые объекты на основе того, что выбирают похожие на него пользователи. Какие здесь могут быть сложности? Что значит "похожесть" в данном случае?

<img src="https://www.researchgate.net/profile/Prakash-Upadhyaya/publication/366902172/figure/fig2/AS:11431281111439652@1672973053129/User-based-and-Item-based-Collaborative-Filtering.png" alt="rec"/>

На практике часто используются комбинированные методы.

### Возможные алгоритмы решения

- Методы на основе векторного расстояния между пользователями / объектами

У нас есть квадратная матрица с рейтингами:

| user_id <br> item_id | 1 | 2 | 3 | 4 | 5 |
| -- | -- | -- | -- | -- | -- |
| 1 | 5.0 | 0.0 | 0.0 | 3.0 | 3.0 |
| 2 | 4.0 | 0.0 | 0.0 | 0.0 | 0.0 |
| 3 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 |
| 4 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 |
| 5 | 4.0 | 3.0 | 0.0 | 0.0 | 0.0 |

Давайте считать, что строки $-$ это вектора пользователей (тогда каждый признак $-$ это оценка для фильма), а столбцы $-$ это вектора объектов (признак $-$ оценка от пользователя). Тогда мы можем находить ближайших пользователей / объекты на основе любого метода подсчета векторной близости (MSE, cosine distance, etc.)

- Методы на основе FCA (Formal Concept Analysis) или pattern mining

Существует группа методов, позволяющих находить множества похожих объектов на основе графового представления данных (например, берем все группы объектов, имеющих связи с одними и теми же вершинами).

Тогда мы считаем похожими объекты или пользователей, оказавшихся в одном множестве.

-  Матричные разложения (и в целом весь спектр методов для задачи восстановления пропусков в матрице)

- NLP подходы (считаем объекты/пользователей токенами и учимся их векторизовать)

## Практика

В питоне есть библиотека `surprise`, в которой собраны многие методы и датасеты для рексистем

In [ ]:
!pip install surprise --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from surprise import NormalPredictor, KNNBasic, KNNWithMeans, SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

import pandas as pd

Прочитаем датасет. Заметим, что это какой-то сложный класс, который не умеет сам себя красиво рисовать

In [ ]:
# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')
data

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


Но мы можем посмотреть на его содержание в сыром виде. Здесь есть четыре колонки:
- Айди пользователя
- Айди объекта (фильма)
- Ретинг, который пользователь поставил фильму
- Время, когда оценка была поставлена

__Зачем нам время?__

В задачах рекомендаций, как и в любой задаче, где есть деление на "сейчас" и "будущее", имеет смысл разделение на обучение и тест делать не случайно, а по времени.

In [ ]:
df = pd.DataFrame(data.raw_ratings,
                  columns=['user_id', 'item_id', 'rating', 'timestamp'])
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


Теперь попробуем решить задачу.

Начнем с `NormalPredictor`. Этот алгоритм просто предсказывает случайный рейтинг на основе того распределения, которое есть в обучающих данных (причем это обучение считается номральным). Воспринимаем это как бейзлайн

In [ ]:
# Создаем класс, у него никаких дополнительных параметров нет
algo = NormalPredictor()

# Запускаем кросс-валидацию
cv_res = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5166  1.5278  1.5157  1.5242  1.5229  1.5215  0.0046  
MAE (testset)     1.2187  1.2304  1.2187  1.2229  1.2208  1.2223  0.0044  
Fit time          0.10    0.13    0.13    0.13    0.13    0.13    0.01    
Test time         0.09    0.25    0.09    0.19    0.08    0.14    0.07    


А еще кросс-вадидация возвращает нам словарь с результатами:

In [ ]:
cv_res

{'test_rmse': array([1.516607  , 1.52781835, 1.5157477 , 1.52420881, 1.5229373 ]),
 'test_mae': array([1.21865444, 1.23041238, 1.21869735, 1.22287405, 1.22075666]),
 'fit_time': (0.10248541831970215,
  0.1312549114227295,
  0.13074636459350586,
  0.13379335403442383,
  0.12691330909729004),
 'test_time': (0.08859682083129883,
  0.25293922424316406,
  0.09192132949829102,
  0.19420146942138672,
  0.08205604553222656)}

Теперь попробуем `KNNBasic` $-$ это алгоритм, который использует векторную близость между объектами / пользователями. У него есть следующие параметры:
- `k` $-$ максимальное кол-во ближашийх объектов, для которых мы усредняем оценки (если всего оценок больше, то все остальные просто игнорируем)
- `min_k` $-$ минимальное кол-во ближашийх объектов, для которых мы усредняем оценки (если меньше, то берем просто среднее по обучающим данным)
- `sim_options` $-$ всякие параметры для функции близости, здесь из важных:
    - `user_based` $-$ `True` или `False`, определяет какой вариант алгоритма мы используем
    - `name` $-$ название метрики близости

Попробуем сначала user-based подход:

In [ ]:
algo = KNNBasic(k=40, min_k=1, sim_options={'user_based': True})
cv_res = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9790  0.9752  0.9817  0.9838  0.9738  0.9787  0.0038  
MAE (testset)     0.7720  0.7704  0.7757  0.7754  0.7703  0.7728  0.0023  
Fit time          0.55    0.46    0.44    0.44    0.49    0.47    0.04    
Test time         3.17    4.09    3.14    3.12    4.38    3.58    0.54    


А теперь item-based:

In [ ]:
algo = KNNBasic(k=40, min_k=1, sim_options={'user_based': False})
cv_res = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9765  0.9743  0.9704  0.9773  0.9747  0.9746  0.0024  
MAE (testset)     0.7710  0.7705  0.7663  0.7733  0.7678  0.7697  0.0025  
Fit time          1.84    0.79    0.70    0.64    0.68    0.93    0.46    
Test time         6.18    4.60    3.63    3.75    4.59    4.55    0.91    


Алгоритм выше во многом хорош, но на практике чаще используется его улучшенная версия. В библиотеке она называется `KNNWithMeans`.

В чем идея улучшений?

Очевидно, что все пользователи имеют у себя в голове немного разную шкалу (кто-то ставит только 5-ки, а кто-то никогда не ставит ничего выше 4-х), так же как и у объекта обычно рейтинги тяготеют к какому-то определенному значению. Чтобы учитывать подобные нюансы, в формулу для усреднения вводится нормирование на срееднее значение по пользователю / объекту.

In [ ]:
algo = KNNWithMeans(k=40, min_k=1, sim_options={'user_based': True})
cv_res = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9485  0.9563  0.9531  0.9457  0.9460  0.9499  0.0041  
MAE (testset)     0.7466  0.7516  0.7510  0.7464  0.7462  0.7484  0.0024  
Fit time          0.47    0.47    0.48    0.87    0.46    0.55    0.16    
Test time         4.22    3.33    3.66    4.63    3.19    3.81    0.55    


Еще можно проверить, как сработает алгоритм, основанный на SVD разложении.

Общая идея:

- Делаем разложение через градиентный спуск на две (!) матрицы, считаем, что одна из них содержит вектора пользователей, а другая $-$ объектов
- Тогда рейтинг $-$ это скалярное произведение вектора пользователя на вектор объекта (собственно, мы матрицу для этого и раскладывали)

Подробнее можно прочитать [тут](https://habr.com/ru/articles/751470/)

In [ ]:
algo = SVD()
cv_res = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9280  0.9477  0.9309  0.9322  0.9372  0.9352  0.0069  
MAE (testset)     0.7323  0.7444  0.7359  0.7367  0.7378  0.7374  0.0040  
Fit time          1.44    1.47    1.86    1.67    1.44    1.58    0.17    
Test time         0.23    0.15    0.20    0.11    0.12    0.16    0.05    
